# Project 1: Titanic Survival Prediction

## Goal
Predict whether a passenger survived the Titanic disaster based on features like Age, Class, Fare, etc.

## Steps
1. Load Data
2. Exploratory Data Analysis (EDA)
3. Data Cleaning
4. Train Model (Logistic Regression)
5. Evaluate

In [1]:
# 1. Install necessary libraries
%pip install pandas seaborn matplotlib scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
# 2. Import Libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset directly from Seaborn
df = sns.load_dataset('titanic')

# View first 5 rows
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


### EDA Checklist:
- Check for missing values: `df.isnull().sum()`
- Visualize survival rate by class: `sns.barplot(x='class', y='survived', data=df)`
- Visualize survival by gender: `sns.barplot(x='sex', y='survived', data=df)`

In [3]:
# Run your EDA code here
print(df.isnull().sum())

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64
